In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


# 1. Data Extraction: Using the BeautifulSoup

In [3]:
from bs4 import BeautifulSoup
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(res,'xml')
table = soup.find('table')

In [4]:
post=[]
borough=[]
neigh=[]
# loop by tag to extract
for tr in table.find_all('tr'):
    i=1
    p1=b1=n1=0
    b1_tag=n1_tag=''
    for td in tr.find_all('td'):
        if i == 1: 
            p1=td.text
        elif i == 2: 
            b1, b1_tag = td.text,td.find('a')
        elif i == 3: 
            n1,n1_tag=str(td.text).strip(),td.find('href')
        i +=1
    # uncomment the print to evaluate filtered out records
    try:
        if ((b1_tag is None) and (n1_tag is None)):
            print ('in try ==>','## b1 is:', b1,'## b1 tag is:', b1_tag,'## n1 is:', n1,'## n1 tag is:', n1_tag)
            continue
    except IndexError:
            continue
    if n1 =='Not assigned' and b1 != 'Not assigned':
        n1=b1
    if p1 != 0:
        post.append(p1), borough.append(b1), neigh.append(n1)

in try ==> ## b1 is: Not assigned ## b1 tag is: None ## n1 is: Not assigned ## n1 tag is: None
in try ==> ## b1 is: Not assigned ## b1 tag is: None ## n1 is: Not assigned ## n1 tag is: None
in try ==> ## b1 is: Not assigned ## b1 tag is: None ## n1 is: Not assigned ## n1 tag is: None
in try ==> ## b1 is: Not assigned ## b1 tag is: None ## n1 is: Not assigned ## n1 tag is: None
in try ==> ## b1 is: Not assigned ## b1 tag is: None ## n1 is: Not assigned ## n1 tag is: None
in try ==> ## b1 is: Not assigned ## b1 tag is: None ## n1 is: Not assigned ## n1 tag is: None
in try ==> ## b1 is: Not assigned ## b1 tag is: None ## n1 is: Not assigned ## n1 tag is: None
in try ==> ## b1 is: York ## b1 tag is: None ## n1 is: Humewood-Cedarvale ## n1 tag is: None
in try ==> ## b1 is: Not assigned ## b1 tag is: None ## n1 is: Not assigned ## n1 tag is: None
in try ==> ## b1 is: Not assigned ## b1 tag is: None ## n1 is: Not assigned ## n1 tag is: None
in try ==> ## b1 is: Not assigned ## b1 tag is: None

# 2. Data Transformation

In [8]:
dict = {'PostalCode':post, 'Borough':borough, 'Neighborhood':neigh}
df = pd.DataFrame.from_dict(dict)
df.to_csv('toronto.csv')

## Print the number of rows of your dataframe

In [7]:
df.shape

(209, 3)

In [5]:
!conda install -c conda-forge geocoder -y


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    orderedset-2.0             |           py36_0         231 KB  conda-forge
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    ratelim-0.1.6              |           py36_0           5 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         288 KB

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0  conda-forge
    orderedset: 2.0-py36_0   conda-forge
    ratelim:    0.1.6-py36_0 conda-forge


orderedset-2.0       | 231 KB    | ##################################### | 100% 
geocoder-1.38.1      | 52 KB     | ##################################### | 100% 
ratelim-0.1.6      

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format('PostalCode'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

#### Unfortunately geocoder didn't work. I used link to a csv file that has the geographical coordinates of each postal code and merged it with previous DataFrame to have proper output

In [23]:
csv_path = 'http://cocl.us/Geospatial_data'
geo = pd.read_csv(csv_path)

In [24]:
geo.columns = ['PostalCode', 'Latitude','Longitude']

In [28]:
geo_df = pd.merge(df, geo,how='outer', on='PostalCode')

In [29]:
geo_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353
